In [1]:
import sys 
sys.path.append("../models/")
from Basin import Basin 
from functions import plot_style
from import_functions import get_averted_deaths
import matplotlib.pyplot as plt
import numpy as np
import os
from statistics import mode
from datetime import datetime
import warnings
warnings.filterwarnings("ignore")

from scipy.stats import pearsonr
colors = plot_style("../fonts/Encode_Sans_Condensed/")
countries = [el.replace("projections_", "") for el in os.listdir("./projections_october_2scaler/") if el  != ".DS_Store"]

end_date = datetime(2021, 10, 1)
vax_start_date = datetime(2020, 12, 14)


# Averted Deaths Respect to Data-Driven Simulation

In [2]:
def import_projections(country, scenario, IQR, baseline='data-driven', path="./projections_october_2scaler/"):

    # get averted deaths in perc
    avertedDeaths_perc = get_averted_deaths(country, scenario, scaled=False, baseline=baseline, perc=True, path=path)
    avertedDeaths_perc_med = np.median(avertedDeaths_perc)
    if IQR:
        avertedDeaths_perc_q1 = np.quantile(avertedDeaths_perc, q=0.25)
        avertedDeaths_perc_q2 = np.quantile(avertedDeaths_perc, q=0.75)
    else:
        avertedDeaths_perc_q1 = np.quantile(avertedDeaths_perc, q=0.05)
        avertedDeaths_perc_q2 = np.quantile(avertedDeaths_perc, q=0.95)

    # get averted deaths raw (without underreporting multiplier)
    avertedDeaths = get_averted_deaths(country, scenario, scaled=False, baseline=baseline, perc=False, path=path)
    avertedDeaths_med = np.median(avertedDeaths)

    # get averted deaths raw (with underreporting multiplier)
    avertedDeaths_scaled = get_averted_deaths(country, scenario, scaled=True, baseline=baseline, perc=False, path=path)
    avertedDeaths_scaled_med = np.median(avertedDeaths_scaled)

    return avertedDeaths_perc_med, avertedDeaths_perc_q1, avertedDeaths_perc_q2, avertedDeaths_med, avertedDeaths_scaled_med
    

def plot_bar(ax, pos, med, q1, q2, color):
    ax.scatter([pos], [med], color=color, s=100, edgecolor='white', linewidth=0.7, zorder=1, marker="o")
    ax.plot([pos, pos], [q1, q2], color=color, solid_capstyle='round', linewidth=8, alpha=0.5, zorder=-1)


def barplot(scenario, IQR, country_title, baseline='data-driven', color=colors[0], path="./projections_october_2scaler/"):
    # get projections
    avertedDeaths_perc_med, avertedDeaths_perc_q1, avertedDeaths_perc_q2 = np.zeros(len(countries)), np.zeros(len(countries)), np.zeros(len(countries))
    avertedDeaths_med, avertedDeaths_scaled_med = np.zeros(len(countries)), np.zeros(len(countries)) 

    for i in range(len(countries)):
        country = countries[i]
        print(country)
        perc_med, perc_q1, perc_q2, med, scaled_med = import_projections(country, scenario, IQR, baseline=baseline, path=path)
        avertedDeaths_perc_med[i] = perc_med
        avertedDeaths_perc_q1[i] = perc_q1
        avertedDeaths_perc_q2[i] = perc_q2
        avertedDeaths_med[i] = med
        avertedDeaths_scaled_med[i] = scaled_med

    # plot  
    fig, axes = plt.subplot_mosaic([["A", "A"], ["A", "A"]], dpi=300, figsize=(9,3))
    idx = np.argsort(avertedDeaths_perc_med)#[::-1]

    for i in range(len(idx)):
        plot_bar(axes["A"], i, avertedDeaths_perc_med[idx[i]], avertedDeaths_perc_q1[idx[i]], avertedDeaths_perc_q2[idx[i]], color)
        axes["A"].annotate(xy=(i, avertedDeaths_perc_q2[idx[i]]+2.7), text=str(int(avertedDeaths_med[idx[i]] / 1000)) + "K", size=7, color="grey", ha="center")

    axes["A"].set_xticks(range(len(idx)))
    axes["A"].set_xticklabels(countries[idx], rotation=90, size=8)
    axes["A"].set_ylabel("Averted Deaths (%)",  fontsize=8)
    if "rescale" in scenario:
        axes["A"].set_title("Averted deaths by {0} vaccination rates".format(country_title), weight="bold")
    elif "start" in scenario:
        axes["A"].set_title("Averted deaths starting vaccinations as {0}".format(country_title), weight="bold")

    for ax in axes.values():
        ax.spines["top"].set_visible(False)
        ax.spines["right"].set_visible(False)
        ax.yaxis.grid(True, color='grey', linewidth=0.1, zorder=-1, linestyle="--")
        ax.set_axisbelow(True)
        ax.tick_params(axis="y", labelsize=8)

    file_name = "averted_deaths_" + scenario
    if IQR:
        file_name += "_IQR"
    
    #plt.savefig("./img/" + file_name + ".pdf", bbox_inches="tight")
    return avertedDeaths_perc_med

In [3]:
#countries = np.array(countries)
#avertedDeaths_perc_med = barplot("us_rescale", IQR=True, baseline='data-driven', color=colors[0], country_title="US", path="./projections_october/")

In [4]:
countries = ["Kenya", "Afghanistan", 'Honduras', 'Cote-dIvoire', 'Kyrgyzstan', 'Zambia',
                'Ghana', 'Sri-Lanka', 'El-Salvador', 'Indonesia', 'Morocco', 'Mozambique',
                'Angola','Bolivia', 'Philippines', 'Bangladesh', 'Rwanda',
                'Egypt', 'Pakistan']

In [17]:
print("US Rescale")

for country in countries:
    print(country)
    avertedDeaths = get_averted_deaths(country, 'us_rescale', scaled=False, baseline='data-driven', perc=True, path='./projections_october_2scaler/')
    med, q1, q2 = np.median(avertedDeaths), np.quantile(avertedDeaths, q=0.25), np.quantile(avertedDeaths, q=0.75)
    print('\t{:.1f} [{:.1f}, {:.1f}]'.format(med, q1, q2))
    print("\n")



US Rescale
Kenya
	71.1 [69.4, 73.4]


Afghanistan
	93.4 [91.2, 95.3]


Honduras
	61.1 [56.2, 66.6]


Cote-dIvoire
	66.8 [63.6, 69.2]


Kyrgyzstan
	78.3 [73.4, 82.6]


Zambia
	73.9 [69.8, 77.9]


Ghana
	63.4 [61.4, 65.2]


Sri-Lanka
	62.8 [59.2, 66.5]


El-Salvador
	59.4 [56.3, 62.3]


Indonesia
	81.5 [78.9, 83.7]


Morocco
	65.7 [60.8, 70.6]


Mozambique
	70.0 [67.3, 72.7]


Angola
	55.2 [54.7, 55.8]


Bolivia
	70.4 [68.3, 72.4]


Philippines
	75.1 [73.7, 76.8]


Bangladesh
	83.7 [80.3, 86.8]


Rwanda
	80.4 [76.2, 84.1]


Egypt
	56.0 [54.3, 57.9]


Pakistan
	70.8 [68.8, 72.5]




In [18]:
print("US Rescale")

for country in countries:
    print(country)
    avertedDeaths = get_averted_deaths(country, 'us_rescale', scaled=False, baseline='data-driven', perc=False, path='./projections_october_2scaler/')
    med, q1, q2 = np.median(avertedDeaths/1000), np.quantile(avertedDeaths/1000, q=0.25), np.quantile(avertedDeaths/1000, q=0.75)
    print('\t{:.1f}K [{:.1f}K, {:.1f}K]'.format(med, q1, q2))
    print("\n")



US Rescale
Kenya
	24.4K [21.0K, 28.7K]


Afghanistan
	16.2K [13.4K, 20.3K]


Honduras
	5.4K [4.7K, 6.3K]


Cote-dIvoire
	13.5K [11.3K, 18.6K]


Kyrgyzstan
	1.6K [1.2K, 2.1K]


Zambia
	11.1K [8.8K, 13.8K]


Ghana
	35.7K [33.1K, 38.6K]


Sri-Lanka
	18.4K [14.9K, 22.4K]


El-Salvador
	2.2K [1.8K, 2.6K]


Indonesia
	137.5K [118.7K, 163.0K]


Morocco
	10.7K [8.8K, 12.9K]


Mozambique
	21.3K [18.0K, 27.1K]


Angola
	12.4K [10.3K, 15.8K]


Bolivia
	7.6K [7.1K, 8.1K]


Philippines
	42.5K [38.0K, 47.6K]


Bangladesh
	66.6K [54.5K, 82.7K]


Rwanda
	1.4K [1.0K, 1.9K]


Egypt
	14.5K [13.1K, 16.2K]


Pakistan
	22.6K [18.3K, 27.9K]




In [19]:
print("US Start")

for country in countries:
    print(country)
    avertedDeaths = get_averted_deaths(country, 'us_start', scaled=False, baseline='data-driven', perc=True, path='./projections_october_2scaler/')
    med, q1, q2 = np.median(avertedDeaths), np.quantile(avertedDeaths, q=0.25), np.quantile(avertedDeaths, q=0.75)
    print('\t{:.1f} [{:.1f}, {:.1f}]'.format(med, q1, q2))
    print("\n")



US Start
Kenya
	19.2 [18.7, 19.7]


Afghanistan
	14.3 [13.2, 15.5]


Honduras
	31.7 [30.0, 33.5]


Cote-dIvoire
	16.5 [15.9, 17.2]


Kyrgyzstan
	36.7 [32.9, 41.0]


Zambia
	22.4 [21.2, 23.5]


Ghana
	14.3 [13.3, 15.1]


Sri-Lanka
	44.4 [43.0, 46.1]


El-Salvador
	46.8 [44.2, 49.5]


Indonesia
	20.7 [19.9, 21.5]


Morocco
	48.0 [42.8, 53.7]


Mozambique
	29.2 [28.3, 30.1]


Angola
	21.6 [21.1, 22.1]


Bolivia
	33.9 [32.6, 35.2]


Philippines
	42.5 [40.8, 44.2]


Bangladesh
	18.2 [15.8, 20.7]


Rwanda
	42.1 [32.8, 51.0]


Egypt
	5.8 [4.8, 6.8]


Pakistan
	22.7 [21.2, 24.1]




In [20]:
print("US Start")

for country in countries:
    print(country)
    avertedDeaths = get_averted_deaths(country, 'us_start', scaled=False, baseline='data-driven', perc=False, path='./projections_october_2scaler/')
    med, q1, q2 = np.median(avertedDeaths/1000), np.quantile(avertedDeaths/1000, q=0.25), np.quantile(avertedDeaths/1000, q=0.75)
    print('\t{:.1f}K [{:.1f}K, {:.1f}K]'.format(med, q1, q2))
    print("\n")



US Start
Kenya
	6.6K [5.6K, 7.9K]


Afghanistan
	2.5K [2.0K, 3.2K]


Honduras
	2.8K [2.5K, 3.2K]


Cote-dIvoire
	3.4K [2.8K, 4.6K]


Kyrgyzstan
	0.8K [0.6K, 1.0K]


Zambia
	3.3K [2.7K, 4.2K]


Ghana
	8.0K [7.2K, 8.8K]


Sri-Lanka
	13.0K [10.5K, 16.0K]


El-Salvador
	1.7K [1.4K, 2.0K]


Indonesia
	35.3K [30.6K, 41.4K]


Morocco
	7.7K [6.4K, 9.4K]


Mozambique
	9.0K [7.5K, 11.5K]


Angola
	4.9K [4.0K, 6.2K]


Bolivia
	3.7K [3.4K, 3.9K]


Philippines
	24.0K [21.5K, 26.8K]


Bangladesh
	14.5K [11.5K, 18.5K]


Rwanda
	0.7K [0.5K, 1.1K]


Egypt
	1.5K [1.2K, 1.9K]


Pakistan
	7.3K [5.9K, 8.9K]




In [10]:
print("US Rescale")

for country in ['Sri-Lanka', 'El-Salvador', 'Morocco', 'Bolivia', 'Honduras',
            'Philippines', 'Indonesia','Pakistan', 'Rwanda', 'Bangladesh',
            'Kyrgyzstan', 'Egypt', 'Mozambique', 'Afghanistan', 'Angola',
            'Ghana', 'Zambia', "Cote-dIvoire", 'Kenya']:
    avertedDeaths_2scaler_perc = get_averted_deaths(country, 'us_rescale', scaled=False, baseline='data-driven', perc=True, path='./projections_october_2scaler/')
    med_2scaler_perc, q1_2scaler_perc, q2_2scaler_perc = np.median(avertedDeaths_2scaler_perc), np.quantile(avertedDeaths_2scaler_perc, q=0.25), np.quantile(avertedDeaths_2scaler_perc, q=0.75)
    med_2scaler_perc, q1_2scaler_perc, q2_2scaler_perc = np.round(med_2scaler_perc, 2), np.round(q1_2scaler_perc, 2), np.round(q2_2scaler_perc, 2)

    avertedDeaths_2scaler = get_averted_deaths(country, 'us_rescale', scaled=False, baseline='data-driven', perc=False, path='./projections_october_2scaler/')
    med_2scaler, q1_2scaler, q2_2scaler = np.median(avertedDeaths_2scaler/1000), np.quantile(avertedDeaths_2scaler/1000, q=0.25), np.quantile(avertedDeaths_2scaler/1000, q=0.75)
    med_2scaler, q1_2scaler, q2_2scaler = np.round(med_2scaler, 1), np.round(q1_2scaler, 1), np.round(q2_2scaler, 1)

    avertedDeaths_perc = get_averted_deaths(country, 'us_rescale', scaled=False, baseline='data-driven', perc=True, path='./projections_october/')
    med_perc, q1_perc, q2_perc = np.median(avertedDeaths_perc), np.quantile(avertedDeaths_perc, q=0.25), np.quantile(avertedDeaths_perc, q=0.75)
    med_perc, q1_perc, q2_perc = np.round(med_perc, 2), np.round(q1_perc, 2), np.round(q2_perc, 2)

    avertedDeaths = get_averted_deaths(country, 'us_rescale', scaled=False, baseline='data-driven', perc=False, path='./projections_october/')
    med, q1, q2 = np.median(avertedDeaths/1000), np.quantile(avertedDeaths/1000, q=0.25), np.quantile(avertedDeaths/1000, q=0.75)
    med, q1, q2 = np.round(med, 1), np.round(q1, 1), np.round(q2, 1) 
    #print('\t{:.1f} [{:.1f}, {:.1f}]'.format(med, q1, q2))

    line = "{0} & {1}K {{[}}{2}K, {3}K{{]}} & {4}K {{[}}{5}K, {6}K{{]}} & {7} {{[}}{8}, {9}{{]}} & {10} {{[}}{11}, {12}{{]}}".format(country,  med, q1, q2, med_2scaler, q1_2scaler, q2_2scaler,  med_perc, q1_perc, q2_perc, med_2scaler_perc, q1_2scaler_perc, q2_2scaler_perc)
    print(line + "\\\ ")



US Rescale
Sri-Lanka & 19.1K {[}15.2K, 25.0K{]} & 18.4K {[}14.9K, 22.4K{]} & 62.61 {[}58.77, 67.05{]} & 62.8 {[}59.2, 66.46{]}\\ 
El-Salvador & 2.2K {[}1.7K, 2.7K{]} & 2.2K {[}1.8K, 2.6K{]} & 59.31 {[}55.94, 63.02{]} & 59.37 {[}56.3, 62.31{]}\\ 
Morocco & 12.1K {[}9.7K, 15.5K{]} & 10.7K {[}8.8K, 12.9K{]} & 67.65 {[}63.59, 71.62{]} & 65.66 {[}60.8, 70.56{]}\\ 
Bolivia & 7.8K {[}7.2K, 8.3K{]} & 7.6K {[}7.1K, 8.1K{]} & 70.76 {[}68.86, 72.59{]} & 70.37 {[}68.26, 72.39{]}\\ 
Honduras & 5.8K {[}4.8K, 7.1K{]} & 5.4K {[}4.7K, 6.3K{]} & 60.91 {[}55.68, 66.17{]} & 61.13 {[}56.23, 66.61{]}\\ 
Philippines & 51.7K {[}43.2K, 61.3K{]} & 42.5K {[}38.0K, 47.6K{]} & 79.61 {[}76.48, 82.61{]} & 75.11 {[}73.68, 76.76{]}\\ 
Indonesia & 149.3K {[}122.5K, 182.7K{]} & 137.5K {[}118.7K, 163.0K{]} & 81.97 {[}78.91, 84.36{]} & 81.51 {[}78.86, 83.71{]}\\ 
Pakistan & 28.3K {[}21.1K, 38.2K{]} & 22.6K {[}18.3K, 27.9K{]} & 71.8 {[}69.93, 73.68{]} & 70.79 {[}68.8, 72.48{]}\\ 
Rwanda & 1.7K {[}1.1K, 2.6K{]} & 1.4K {[}1.

In [11]:
print("US Start")

for country in ['Sri-Lanka', 'El-Salvador', 'Morocco', 'Bolivia', 'Honduras',
            'Philippines', 'Indonesia','Pakistan', 'Rwanda', 'Bangladesh',
            'Kyrgyzstan', 'Egypt', 'Mozambique', 'Afghanistan', 'Angola',
            'Ghana', 'Zambia', "Cote-dIvoire", 'Kenya']:
    avertedDeaths_2scaler_perc = get_averted_deaths(country, 'us_start', scaled=False, baseline='data-driven', perc=True, path='./projections_october_2scaler/')
    med_2scaler_perc, q1_2scaler_perc, q2_2scaler_perc = np.median(avertedDeaths_2scaler_perc), np.quantile(avertedDeaths_2scaler_perc, q=0.25), np.quantile(avertedDeaths_2scaler_perc, q=0.75)
    med_2scaler_perc, q1_2scaler_perc, q2_2scaler_perc = np.round(med_2scaler_perc, 2), np.round(q1_2scaler_perc, 2), np.round(q2_2scaler_perc, 2)

    avertedDeaths_2scaler = get_averted_deaths(country, 'us_start', scaled=False, baseline='data-driven', perc=False, path='./projections_october_2scaler/')
    med_2scaler, q1_2scaler, q2_2scaler = np.median(avertedDeaths_2scaler/1000), np.quantile(avertedDeaths_2scaler/1000, q=0.25), np.quantile(avertedDeaths_2scaler/1000, q=0.75)
    med_2scaler, q1_2scaler, q2_2scaler = np.round(med_2scaler, 1), np.round(q1_2scaler, 1), np.round(q2_2scaler, 1)

    avertedDeaths_perc = get_averted_deaths(country, 'us_start', scaled=False, baseline='data-driven', perc=True, path='./projections_october/')
    med_perc, q1_perc, q2_perc = np.median(avertedDeaths_perc), np.quantile(avertedDeaths_perc, q=0.25), np.quantile(avertedDeaths_perc, q=0.75)
    med_perc, q1_perc, q2_perc = np.round(med_perc, 2), np.round(q1_perc, 2), np.round(q2_perc, 2)

    avertedDeaths = get_averted_deaths(country, 'us_start', scaled=False, baseline='data-driven', perc=False, path='./projections_october/')
    med, q1, q2 = np.median(avertedDeaths/1000), np.quantile(avertedDeaths/1000, q=0.25), np.quantile(avertedDeaths/1000, q=0.75)
    med, q1, q2 = np.round(med, 1), np.round(q1, 1), np.round(q2, 1) 
    #print('\t{:.1f} [{:.1f}, {:.1f}]'.format(med, q1, q2))

    line = "{0} & {1}K {{[}}{2}K, {3}K{{]}} & {4}K {{[}}{5}K, {6}K{{]}} & {7} {{[}}{8}, {9}{{]}} & {10} {{[}}{11}, {12}{{]}}".format(country,  med, q1, q2, med_2scaler, q1_2scaler, q2_2scaler,  med_perc, q1_perc, q2_perc, med_2scaler_perc, q1_2scaler_perc, q2_2scaler_perc)
    print(line + "\\\ ")



US Start
Sri-Lanka & 13.5K {[}10.7K, 17.7K{]} & 13.0K {[}10.5K, 16.0K{]} & 44.29 {[}42.44, 46.39{]} & 44.4 {[}42.96, 46.11{]}\\ 
El-Salvador & 1.7K {[}1.4K, 2.2K{]} & 1.7K {[}1.4K, 2.0K{]} & 46.86 {[}43.62, 50.54{]} & 46.78 {[}44.22, 49.49{]}\\ 
Morocco & 9.0K {[}7.1K, 11.6K{]} & 7.7K {[}6.4K, 9.4K{]} & 50.25 {[}45.17, 55.55{]} & 48.05 {[}42.77, 53.7{]}\\ 
Bolivia & 3.8K {[}3.5K, 4.0K{]} & 3.7K {[}3.4K, 3.9K{]} & 34.14 {[}32.81, 35.48{]} & 33.88 {[}32.55, 35.19{]}\\ 
Honduras & 3.1K {[}2.5K, 3.7K{]} & 2.8K {[}2.5K, 3.2K{]} & 31.94 {[}30.37, 33.57{]} & 31.72 {[}30.04, 33.5{]}\\ 
Philippines & 29.4K {[}24.9K, 34.1K{]} & 24.0K {[}21.5K, 26.8K{]} & 45.02 {[}41.61, 48.56{]} & 42.5 {[}40.8, 44.24{]}\\ 
Indonesia & 38.0K {[}31.5K, 45.7K{]} & 35.3K {[}30.6K, 41.4K{]} & 20.61 {[}19.69, 21.73{]} & 20.67 {[}19.88, 21.54{]}\\ 
Pakistan & 9.2K {[}6.8K, 12.1K{]} & 7.3K {[}5.9K, 8.9K{]} & 23.08 {[}21.63, 24.51{]} & 22.71 {[}21.24, 24.07{]}\\ 
Rwanda & 0.9K {[}0.5K, 1.4K{]} & 0.7K {[}0.5K, 1.1K{]} & 4

In [17]:
avertedDeaths = get_averted_deaths("Ghana", 'us_rescale', scaled=False, baseline='data-driven', perc=False, path='./projections_october/')
med, q1, q2 = np.median(avertedDeaths/1000), np.quantile(avertedDeaths/1000, q=0.05), np.quantile(avertedDeaths/1000, q=0.95)
med, q1, q2 = np.round(med, 1), np.round(q1, 1), np.round(q2, 1) 
print('\t{:.1f}K [{:.1f}K, {:.1f}K]'.format(med, q1, q2))

	20.0K [11.6K, 34.8K]
